In [1]:
%matplotlib notebook
from util import printColAssets, printSummary, User, get_truncated_normal
import matplotlib.pyplot as plt
import multiprocessing as mp
from simulation_util import *
import sys
import numpy as np
from collections import defaultdict
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Plotter Functions

In [2]:
def plot3D(x, y, z, xtitle, ytitle, ztitle):
    fig = go.Figure(data=[
            go.Scatter3d(x=x, 
                         y=y, 
                         z=z,
                        line=dict(
                            color='darkblue',
                            width=2
                        ) 
                    )])
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
def processData(x,y,z):
    # Get x,y,z sorted based on y i.e TXF
    sortxyz = [(x,y,z) for y,x,z in sorted(zip(y,x,z))]
    
    x = [i[0] for i in sortxyz]
    y = [i[1] for i in sortxyz]
    z = [i[2] for i in sortxyz]
    
    return x,y,z
    
    
def plot3DWithTXF(x,y,z,xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    # Get x,y,z sorted based on y i.e TXF
    x,y,z = processData(x,y,z)
    
    txDictx = defaultdict(list)
    txDictz = defaultdict(list)
    
    for i in range(len(y)):
        txDictx[y[i]].append(x[i])
        txDictz[y[i]].append(z[i])
        
    data = []   
 
              
    cnt = 0          
    for key in txDictx.keys():
        
        x_d = txDictx[key]
        y_d = [key for i in range(len(txDictx[key]))]
        z_d = txDictz[key]

        temp = go.Scatter3d(x=x_d, 
                         y=y_d, 
                         z=z_d,
                        line=dict(
                            color=colors[cnt % len(colors)],
                            width=2
                        ) 
                    )
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
    
def plot2D(x, y, xtitle, ytitle):    
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers'))
    
    fig.update_layout(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
    )
    
    plotly.offline.iplot(fig)


# Simulation Functions

In [3]:
def runOnThread(sample_size, assets, risk_params, cdpRate, txFee, tests, stddev=False):
    avgMarketDAI = 0
    avgDAIPrice = 0

    history = []

    for i in range(tests):
        s = Simulator(rho=2.5, cdpRate=cdpRate, txf=txFee, eth_price=140, sample_size=sample_size,
                      initial_distribution=assets, risk_params=risk_params)
        a, b = s.runSimulation()
        avgDAIPrice += a
        avgMarketDAI += b

        if stddev:
            history.append(a)

    avgDAIPrice /= tests
    avgMarketDAI /= tests

    if stddev:
        print("CDP Rate:" , cdpRate, "TxFee: ", txFee)
        print("Average: ", avgDAIPrice)
        print("stddev: ", np.std(np.array(history), dtype=np.float64))

    return avgDAIPrice

In [8]:
def runTests(sample_size, cdpRates, txf, tests):
    
    # Get number of CPUs
    cpus = mp.cpu_count()
    poolCount = cpus * 2

    # Process pool to handle k simulations/process!
    pool = mp.Pool(processes=poolCount)

    args = []
    
    # Define initial allocation and risk distribution
    assets = getAssets(sample_size, "uniform")
    risk_params = getRiskParams(sample_size)

    for txFee in txf:
        for cdpRate in cdpRates:
            args.append((sample_size, assets, risk_params, cdpRate, txFee, tests))
            
    results = pool.starmap(runOnThread, args)

    # Plot results here
    cdp_axis = [args[i][3] for i in range(len(args))]
    txf_axis = [args[i][4] for i in range(len(args))]
    dai_axis = results
    
    print("Now plotting ....")
    plot3DWithTXF(cdp_axis, txf_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")



# Transaction Fee varied, CDP Rate Varied

In [7]:
cdpRates = [0.01 * i for i in range(1, 35)]
txf = [0.01 * i for i in range(1, 15)]
tests = 1

runTests(10, cdpRates, txf, tests)

Now plotting ....


# Tests to check price settling variance

In [12]:
def runOnThreadVariance(sample_size, cdpRate, txFee, tests):
    avgDAIPrice = 0

    history = []

    for i in range(tests):
        assets = getAssets(sample_size, "normal")
        risk_params = getRiskParams(sample_size)
        s = Simulator(rho=2.5, cdpRate=cdpRate, txf=txFee, eth_price=140, sample_size=sample_size,
                      initial_distribution=assets, risk_params=risk_params)
        a, _ = s.runSimulation()
        avgDAIPrice += a
        history.append(a)

    avgDAIPrice /= tests

    return avgDAIPrice, np.std(np.array(history), dtype=np.float64)

def runTestsVariance(sample_size, cdpRates, tcf, tests):
    
    # Get number of CPUs
    cpus = mp.cpu_count()
    poolCount = cpus * 2

    # Process pool to handle k simulations/process!
    pool = mp.Pool(processes=poolCount)

    args = []

    for txFee in txf:
        for cdpRate in cdpRates:
            args.append((sample_size, cdpRate, txFee, tests))

    results = pool.starmap(runOnThreadVariance, args)

    # Plot results here
    cdp_axis = [args[i][1] for i in range(len(args))]
    txf_axis = [args[i][2] for i in range(len(args))]
 
    return cdp_axis, txf_axis, results

In [15]:
cdpRates = [0.01 * i for i in range(1, 2)]
txf = [0.01 * i for i in range(1, 2)]
tests = 6

cdp_axis, txf_axis, results = runTestsVariance(10, cdpRates, txf, tests)

for i in range(len(results)):
    print("CDP Rate:" , cdp_axis[i], "TxFee: ", txf_axis[i])
    print("Average: ", results[i][0])
    print("stddev: ", results[i][1])
    print()

ValueError: Parameter value must be nonnegative.

### Standard Deviation across multiple tests seems to lie withing $0.21


# Test with initial assset distribution as 100 USD or 100 DAI

In [9]:
cdpRates = [0.01 * i for i in range(1, 35)]
txf = [0.01 * i for i in range(0, 15)]
tests = 1

runTests(10, cdpRates, txf, tests)

Now plotting ....


# Variance Test with 100 USD 100 DAI

In [ ]:
cdpRates = [0.01 * i for i in range(1, 10)]
txf = [0.01 * i for i in range(0, 10)]
tests = 10

cdp_axis, txf_axis, results = runTestsVariance(10, cdpRates, txf, tests)

for i in range(len(results)):
    print("CDP Rate:" , cdp_axis[i], "TxFee: ", txf_axis[i])
    print("Average: ", results[i][0])
    print("stddev: ", results[i][1])
    print()

### Standard Dev across tests is under 0.02

## Test for n = 100

In [ ]:
cdpRates = [0.01 * i for i in range(1,2)]
txf = [0.01 * i for i in range(3,4)]
tests = 1

runTests(100, cdpRates, txf, tests)

In [ ]:
cdpRates = [0.01 * i for i in range(1,6)]
txf = [0.01 * i for i in range(3,7)]
tests = 1

runTests(10, cdpRates, txf, tests)

In [ ]:
cdpRates = [0.01 * i for i in range(1,6)]
txf = [0.01 * i for i in range(3,7)]
tests = 1

runTests(200, cdpRates, txf, tests)